In [9]:
import numpy as np
import time
import glob
import os, sys
# from sklearn.metrics import confusion_matrix 
# from sklearn.metrics import accuracy_score 
# from sklearn.metrics import classification_report 
from easydict import EasyDict
sys.path.append('..')
%pylab inline
from util.filters import filter_2d,make_gaussian_kernel
from util.image import convert_to_grayscale

program_start = time.time()
dataset_names = ['easy', 'medium_1', 'medium_2', 'hard']
weights = [0.5, 0.2, 0.2, 0.1]
classes = ['ball', 'brick', 'cylinder']
data_path = '../data' 

dataset_weights = {}
for i in range(len(dataset_names)):
    dataset_weights[dataset_names[i]] = weights[i]

    #Store data performance data in a nested easydict:
    performance = EasyDict()
    total_time = 0

balls_mean = []
balls_max = []
balls_ratio = []
cylinder_mean = []
cylinder_max = []
cylinder_ratio = []
brick_mean = []
brick_max = []
brick_ratio = []
for dataset_name in dataset_names:
    performance[dataset_name] = EasyDict()
    performance[dataset_name]['overall'] = np.array([0, 0]) #overall Correct/incorrect

    for image_class in classes:
        performance[dataset_name][image_class] = np.array([0, 0])
        image_paths = glob.glob(data_path + '/' 
                                + dataset_name + '/' 
                                + image_class + '/*.jpg')

        for image_path in image_paths:
            im = imread(image_path)
#             print(im.shape)
            # im = imread('../data/easy/brick/brick_2.jpg')
            im = im[10:245,10:245]
            start_time = time.time()
            gray = convert_to_grayscale(im/255.)
            # plt.imshow(gray, cmap = 'gray')
            Kx = np.array([[1, 0, -1],
                           [2, 0, -2],
                           [1, 0, -1]])

            Ky = np.array([[1, 2, 1],
                           [0, 0, 0],
                           [-1, -2, -1]])

            # gray = filter_2d(gray,gaussian_kernel)

            Gx = filter_2d(gray, Kx)
            Gy = filter_2d(gray, Ky)

            G_magnitude = np.sqrt(Gx**2+Gy**2)
#             G_direction = np.arctan2(Gy, Gx)
            G_mean = np.mean(G_magnitude)
#             imshow(G_magnitude)
#             colorbar()
            labels = ['brick', 'ball', 'cylinder']
            
            G_max = np.max(G_magnitude)
#             limits = 0. * G_max
            difference = G_max - G_mean
            y_coords, x_coords = np.where(G_magnitude > difference)
            hist_values = np.histogram(G_magnitude)
            
#             len_hist_values = len(hist_values)
            sum_hist_values = 0

            for i in range(int((len(hist_values))/2) + 1,len(hist_values[0])):
                sum_hist_values += hist_values[0][i]*hist_values[1][i]
#             print("hist_values",hist_values)
            
#             if difference > 0 and difference <=1.72:
#             if difference > 0 and difference <=1.72:
#                 prediction = labels[1]
#             elif difference > 1.72 and difference <= 2.045:
#                 prediction = labels[2]
#             else:
#                 prediction = labels[0]
            if sum_hist_values > 0 and sum_hist_values <=1500:
                prediction = labels[1]
            elif sum_hist_values > 1500 and sum_hist_values <= 1750:
                prediction = labels[2]
            else:
                prediction = labels[0]
                    
#             if len(x_coords) > 900 and len(x_coords) <= 1600:
#                 prediction = 'cylinder'
#                 print(labels[2])
                
#             elif len(x_coords) > 0 and len(x_coords) <= 900:
#                 prediction = 'ball'
#                 print(labels[1])
#             else:
#                 prediction = 'brick'
#                 print(labels[0])
                
#             hist_values = np.histogram(G_direction)[0]
#             checkpoint_1 = 0.6 * hist_values.mean()
            
#             if hist_values.mean()<50:
#                 return labels[0]
#             elif hist_values.std() > checkpoint_1 or hist_values.std() > 110:
#                 return labels[1]
#             else:
#                 return labels[2]
            end_time = time.time()
#             if image_class == 'ball':
#                 balls_mean.append(G_mean)
#                 balls_max.append(G_max)
# #                 balls_ratio.append(sum_of_values/len(x_coords))
#             if image_class == 'cylinder':
#                 cylinder_mean.append(G_mean)
#                 cylinder_max.append(G_max)
# #                 cylinder_ratio.append(sum_of_values/len(x_coords))
#             if image_class == 'brick':
#                 brick_mean.append(G_mean)
#                 brick_max.append(G_max)
#                 brick_ratio.append(sum_of_values/len(x_coords)) 
            total_time += (end_time - start_time)
            print(str(image_path),image_class,prediction,G_mean,G_max,G_max-G_mean,sum_hist_values,np.max(hist_values[1]))
        
            if image_class == prediction:
                performance[dataset_name][image_class][0] += 1
            else:
                performance[dataset_name][image_class][1] += 1
                
        #Tally overall performance for class
        performance[dataset_name]['overall'] = performance[dataset_name]['overall'] \
                                               + performance[dataset_name][image_class]
            # print(hist_values)
            # # checkpoint_1 = 0.6 * hist_values.mean()

            # if hist_values.mean()<50:
            #     print(labels[0])
            # elif hist_values.std() > checkpoint_1 or hist_values.std() > 110:
            #     print (labels[1])
            # else:
            #     print (labels[2])

            # y_coords, x_coords = np.where(edges)


                
# print("-------------Balls-------------")
# print(np.sum(balls_mean))
# print(np.sum(balls_max))
# # print(balls_ratio)
# print("-------------Cylinder-------------")
# print(np.sum(cylinder_mean))
# print(np.sum(cylinder_max))
# # print(cylinder_ratio)
# print("-------------Bricks-------------")
# print(np.sum(brick_mean))
# print(np.sum(brick_max))
# print(brick_ratio)
    

#Print out errors: 
print('Fraction of Correct Predictions: ')
overall_accuracy = 0
for dataset_name in dataset_names:
    total_correct, total_incorrect = performance[dataset_name]['overall']
    total = total_correct + total_incorrect
    accuracy = np.round(float(total_correct)/total, 4)
    overall_accuracy += accuracy * dataset_weights[dataset_name]
    performance[dataset_name]['accuracy'] = accuracy
    print(dataset_name + ': ' + \
           '(' + str(total_correct) + '/' + str(total) + ' = ' + \
           str(accuracy) + ')')

    for image_class in classes:
        print('   ' + image_class + ': '
              + str(performance[dataset_name][image_class][0])
              + '/'
              + str(np.sum(performance[dataset_name][image_class])))

overall_accuracy = np.round(overall_accuracy, 2)
score = 0

if overall_accuracy >= 0.6:
   score = 10
elif overall_accuracy >= 0.55:
   score = 9
elif overall_accuracy >= 0.5:
   score = 8
elif overall_accuracy >= 0.45:
   score = 7
elif overall_accuracy >= 0.4:
   score = 6
elif overall_accuracy >= 0.35:
   score = 5
elif overall_accuracy >= 0:
   score = 4


print("\nOverall Accuracy = ", overall_accuracy)
print("Score = ", score)

program_end = time.time()
complete_time = program_end - program_start
total_time =  round(total_time, 5)
complete_time = round(complete_time, 5)

# print("Classification time (seconds): = ", total_time)
print("Program completetion time (seconds): = ", complete_time)


#Compute Gradient Magnitude and Direction:
# G_magnitude = np.sqrt(Gx**2+Gy**2)
#     G_direction = np.arctan2(Gy, Gx)

#     edges = np.zeros(G_magnitude.shape)*np.NaN #Create empty array of NaNs
#     #Replace pixels with gradient estimates above thresh with the direction of the gradient estimate:
#     edges[G>1.15] = G_direction[G_magnitude>thresh] 
# labels = ['brick', 'ball', 'cylinder']

# print(len(x_coords))
# if len(x_coords) > 0 and len(x_coords) <= 1600:
#     return labels[0]
# elif len(x_coords) > 1600 and len(x_coords) <= 2500:
#     return labels[1]
# elif len(x_coords) > 2500:
#     return labels[2]

# imshow(edges)

Populating the interactive namespace from numpy and matplotlib
../data/easy/ball\ball_1.jpg ball ball 0.051650788236792304 1.568027118238159 1.5163763300013668 715.0203659166006 1.568027118238159
../data/easy/ball\ball_2.jpg ball ball 0.058339924175717754 1.452973958744482 1.3946340345687644 868.2972377457024 1.452973958744482
../data/easy/ball\ball_3.jpg ball ball 0.07691831358483557 1.5353830139790958 1.4584647003942603 1098.1059315978493 1.5353830139790958
../data/easy/ball\ball_4.jpg ball ball 0.08442733085427415 1.6177176983322175 1.5332903674779432 1309.0571614904304 1.6177176983322175
../data/easy/ball\ball_5.jpg ball ball 0.0551167209356615 1.3962962792995006 1.341179558363839 885.2518410758834 1.3962962792995006
../data/easy/ball\ball_6.jpg ball cylinder 0.13522921587704045 1.853024977644144 1.7177957617671036 1614.9112680168714 1.853024977644144
../data/easy/ball\ball_7.jpg ball ball 0.05437875112223376 1.7591643809108108 1.704785629788577 705.9526660595085 1.7591643809108108

../data/hard/brick\brick_3.jpg brick brick 0.17106630463978162 2.6444215055269815 2.4733552008872 2086.977452161894 2.6444215055269815
../data/hard/brick\brick_4.jpg brick brick 0.19494974139607207 2.161976903807849 1.967027162411777 3266.7471016536597 2.161976903807849
../data/hard/brick\brick_5.jpg brick brick 0.19624652923976743 3.166170125042508 2.9699235958027406 3560.041688597795 3.166170125042508
../data/hard/brick\brick_6.jpg brick brick 0.20669171102035644 3.1417946456803336 2.935102934659977 3361.720270877957 3.1417946456803336
../data/hard/brick\brick_7.jpg brick brick 0.244527638264658 3.177301056075708 2.93277341781105 3484.5460681982295 3.177301056075708
../data/hard/brick\brick_8.jpg brick brick 0.2738371735175006 3.7575103712220828 3.483673197704582 4869.733441103819 3.7575103712220828
../data/hard/cylinder\cylinder_1.jpg cylinder brick 0.18384035914719773 1.6400233596212523 1.4561830004740546 4742.291546680813 1.6400233596212523
../data/hard/cylinder\cylinder_2.jpg cyl

In [2]:
# plt.imshow(G_magnitude)
# colorbar()

# y_coords, x_coords = np.where(edges)

###### 